In [19]:
import pandas as pd
import numpy as np
import torch
import csv
from torch import nn
from ast import literal_eval
import torch.optim as optim

In [20]:
#Import Samples
samples = pd.read_csv('Sample.csv')

In [21]:
#Define the network
class Linear(nn.Module):
    def __init__(self):
        super(Linear, self).__init__()
        #coefficient
        self.a = torch.nn.Parameter(torch.rand(1, requires_grad=True))
        #slope
        self.b = torch.nn.Parameter(torch.rand(1, requires_grad=True))
        
    def forward(self, feature):
        #Set up LSTM model 
        input = torch.reshape(torch.tensor(feature),[5,1,5])
        rnn = nn.LSTM(5, 20, 2)
        h0 = torch.randn(2, 1, 20)
        c0 = torch.randn(2, 1, 20)
        output, (hn, cn) = rnn(input, (h0, c0))
        
        #Applies a linear transformation to the incoming data
        x = output[-1]
        m = nn.Linear(20, 1)
        linear_x = m(x)
        return self.a*linear_x+self.b
linear = Linear()

In [22]:
#Training Data
for i in range(0,len(samples)):
    #Get sample
    sample = samples.iloc[i]
    feature = literal_eval(sample['Feature'])
    label = sample['Label']
    
    #Apply Linear() function
    new_label = linear(feature)
    linear.zero_grad()
    new_label.backward(torch.randn(1,1))
    
    #Loss Function
    new_label = linear(feature)
    label = torch.Tensor([label])
    label = label.view(1,-1)
    criterion = nn.MSELoss()
    loss = criterion(new_label, label)
    
    #Backprop
    linear.zero_grad()    
    loss.backward()
    
    # create optimizer
    optimizer = optim.SGD(linear.parameters(), lr=0.01)
    optimizer.zero_grad()   # zero the gradient buffers
    new_label = linear(feature)
    loss = criterion(new_label, label)
    loss.backward()
    optimizer.step()    # Does the update

In [24]:
print(linear.a)
print(linear.b)

Parameter containing:
tensor([0.1058], requires_grad=True)
Parameter containing:
tensor([-0.0157], requires_grad=True)
